In [2]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [3]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#导入score_train数据
score_train_name=['id','college','score']
score_train=pd.read_table(path+'/train/score_train.txt',sep=',',header=None,names=score_train_name)

#删除重复行并浏览前几行数据
score_train=score_train.drop_duplicates()
score_train.head()

,id,college,score
0,0,9,1
1,1,9,2
2,8,6,1565
3,9,6,1570
4,10,3,1


In [13]:
score_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9130 entries, 0 to 9129
Data columns (total 3 columns):
id         9130 non-null int64
college    9130 non-null int64
score      9130 non-null int64
dtypes: int64(3)
memory usage: 285.3 KB


In [4]:
subsidy_feature=pd.read_csv('all_feature_input/subsidy_train_feature.csv')
score_train=pd.merge(subsidy_feature,score_train,how='left',on='id')

#缺失填充
score_train['college']=score_train['college'].fillna(method='pad')

In [5]:
#获得每个学院的总人数

college_num=DataFrame(score_train.groupby('college')['score'].max())

In [6]:
#导出数据（不用college做index）

college_num.to_csv('score_input/college_train_num.csv',index=True)

In [7]:
#读取数据

college_num=pd.read_csv('score_input/college_train_num.csv')
college_num.columns=['college','college_num']

In [8]:
#拼接两表

score_train=pd.merge(score_train,college_num,how='left',on='college')

In [9]:
#填充缺失的score，分学院填充

def fill_na_score(df):
    a=list(df.groupby('college')['id'].count().index)
    for i in range(len(a)):
        if i==0:
            college=df.loc[df['college']==a[i]]
            college['score']=college['score'].fillna(method='pad')
        else:
            college_i=df.loc[df['college']==a[i]]
            college_i['score']=college_i['score'].fillna(method='pad')
            college=pd.concat([college,college_i])
    return college

#运用函数
score_train=fill_na_score(score_train)       

C:\Users\Wvv\Anaconda3\envs\py2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Wvv\Anaconda3\envs\py2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
#计算该生在学院排名的百分百

score_train['score_order']=score_train['score'].divide(score_train['college_num'],fill_value=0)

In [21]:
score_train.head()

,id,money,college,score,college_num,score_order
0,0,0,9.0,1.0,2933.0,0.000341
1,1,0,9.0,2.0,2933.0,0.000682
2,8,0,6.0,1565.0,1570.0,0.996815
3,9,0,6.0,1570.0,1570.0,1.000000
4,10,0,3.0,1.0,2277.0,0.000439


In [12]:
len(score_train)

10885

In [10]:
score=score_train[score_train['score'].isnull()]
score

,id,money,college,score,college_num


In [13]:
score_train.drop('money',axis=1,inplace=True)

In [14]:
score_train.to_csv('score_input/score_train_feature.csv',index=False)